In [1]:
from utils import *
import numpy as np
import matplotlib.pyplot as plt
import pyqpanda as pq
import pyvqnet as pv
import random
from functools import partial
import time

%matplotlib inline

In [2]:
train_data = Dataset(
    "data/cifar-10-batches-py/data_batch_1",
    "data/cifar-10-batches-py/data_batch_2",
    "data/cifar-10-batches-py/data_batch_3",
    "data/cifar-10-batches-py/data_batch_4",
    "data/cifar-10-batches-py/data_batch_5",
)
train_data.enhance()
test_data = Dataset("data/cifar-10-batches-py/test_batch")

In [3]:
class qm(pv.qnn.vqc.QModule):
    def __init__(self, name=""):
        super().__init__(name)
        self.device = pv.qnn.vqc.QMachine(2)
        self.RX1 = pv.qnn.vqc.qcircuit.RX(wires=0)
        self.RX2 = pv.qnn.vqc.qcircuit.RX(wires=1)
        self.RY1 = pv.qnn.vqc.qcircuit.RY(True, True, 0, wires=0)
        self.RY2 = pv.qnn.vqc.qcircuit.RY(True, True, 0, wires=1)
        self.RX3 = pv.qnn.vqc.qcircuit.RX(True, True, 0, wires=0)
        self.RX4 = pv.qnn.vqc.qcircuit.RX(True, True, 0, wires=1)
        self.CZ1 = pv.qnn.vqc.qcircuit.CZ(wires=(0, 1))
        self.CZ2 = pv.qnn.vqc.qcircuit.CZ(wires=(1, 0))
        self.RY3 = pv.qnn.vqc.qcircuit.RY(True, True, 0, wires=0)
        self.RY4 = pv.qnn.vqc.qcircuit.RY(True, True, 0, wires=1)
        self.measure = pv.qnn.vqc.MeasureAll(
            obs=[
                {
                    "wires": [0],
                    "observables": ["Z"],
                    "coefficient": [1],
                }
            ]
        )

    # @partial(pv.qnn.vqc.wrapper_compile)
    def forward(self, x, y):
        self.device.reset_states(x.shape[0])
        self.RX1(q_machine=self.device, params=x[:])
        self.RX2(q_machine=self.device, params=y[:])
        self.RY1(q_machine=self.device)
        self.RY2(q_machine=self.device)
        self.RX3(q_machine=self.device)
        self.RX4(q_machine=self.device)
        self.CZ1(q_machine=self.device)
        self.CZ2(q_machine=self.device)
        self.RY3(q_machine=self.device)
        self.RY4(q_machine=self.device)

        return self.measure(q_machine=self.device)


class QM(pv.nn.module.Module):
    def __init__(self, name=""):
        super().__init__(name)
        self.vqc = qm(name)

    def forward(self, x, y):
        X = pv.tensor.flatten(x, 1)
        Y = pv.tensor.flatten(y, 1)
        Z = pv.tensor.zeros_like(X)
        for i in range(X.shape[1]):
            Z[:, i] = self.vqc(X[:, i], Y[:, i])
        z = pv.tensor.reshape(Z, x.shape)
        return z


class Model(pv.nn.module.Module):
    def __init__(self):
        super(Model, self).__init__()
        self.conv1 = pv.nn.Conv2D(3, 4, (3, 3), (1, 1), "same")
        self.pool1 = pv.nn.MaxPool2D((4, 4), (4, 4))
        self.conv3 = pv.nn.Conv2D(4, 8, (3, 3), (1, 1), "same")
        self.pool3 = pv.nn.MaxPool2D((4, 4), (4, 4))
        self.fc = pv.nn.Linear(8 * 2 * 2, 10)

    def forward(self, x):
        x = 2 * pv.tensor.atan(self.pool1(self.conv1(x)))
        x = 2 * pv.tensor.atan(self.pool3(self.conv3(x)))
        x = pv.tensor.flatten(x, 1)
        x = self.fc(x)

        return x

In [4]:
epoch = 1000
batch = 32
holdout = 10000

model = Model()
#model.load_state_dict(pv.utils.storage.load_parameters("train19.model"))
print(pv.model_summary(model))

e = 1
acc_train = []
acc_test = []



###################Model Summary#######################

classic layers: {'Conv2D': 2, 'MaxPool2D': 2, 'Linear': 1}
total classic parameters: 738

qubits num: 0
gates: {}
total quantum gates: 0
total quantum parameter gates: 0
total quantum parameters: 0
#########################################################
    


In [5]:
# batch = 128

In [6]:
X, Y = train_data.getdatas()
los = pv.nn.loss.CrossEntropyLoss()
opt = pv.optim.SGD(model.parameters())
# opt = pv.optim.Adam(model.parameters())
start_time = time.time()

while e <= epoch:
    model.train()

    correct_tot = 0
    for I, (x, y) in enumerate(
        pv.data.data_generator(
            X[:-holdout], Y[:-holdout], batch_size=batch, shuffle=True
        )
    ):
        i = I + 1
        # print(i)
        opt.zero_grad()
        y_pred = model(x)
        loss = los(y, y_pred)
        correct = np.sum(y_pred.argmax(1, False).to_numpy() == y)
        acc = correct / y.shape[0]
        correct_tot += correct
        loss.backward()
        opt._step()
        if i % 100 == 0:
            end_time = time.time()
            print(
                f"epoch {e}/{epoch} \t batch {batch*i}/{Y.shape[0]-holdout} \t loss {loss.item():.2f} \t accuracy {acc:.2f}    \t {end_time-start_time:.2f} s"
            )
            #pv.utils.storage.save_parameters(model.state_dict(), "train16.model")
    e += 1

    model.eval()

    acc = correct_tot / (Y.shape[0] - holdout)
    print(f"********** train: epoch {e}/{epoch} \t accuracy {100*acc:.2f}% **********")
    acc_train.append(acc)

    correct_tot = 0
    for x, y in pv.data.data_generator(
        X[-holdout:], Y[-holdout:], batch_size=128, shuffle=False
    ):
        y_pred = model(x)
        correct_tot += np.sum(y_pred.argmax(1, False).to_numpy() == y)

    acc = correct_tot / holdout
    print(f"********** test : epoch {e}/{epoch} \t accuracy {100*acc:.2f}% **********")
    acc_test.append(acc)

    opt = pv.optim.SGD(model.parameters(), 0.01 * (1 - acc))

epoch 1/1000 	 batch 3200/90000 	 loss 2.27 	 accuracy 0.09    	 0.74 s
epoch 1/1000 	 batch 6400/90000 	 loss 2.21 	 accuracy 0.22    	 1.43 s
epoch 1/1000 	 batch 9600/90000 	 loss 2.22 	 accuracy 0.16    	 1.98 s
epoch 1/1000 	 batch 12800/90000 	 loss 2.33 	 accuracy 0.19    	 2.58 s
epoch 1/1000 	 batch 16000/90000 	 loss 2.18 	 accuracy 0.28    	 3.23 s
epoch 1/1000 	 batch 19200/90000 	 loss 2.02 	 accuracy 0.34    	 3.83 s
epoch 1/1000 	 batch 22400/90000 	 loss 2.10 	 accuracy 0.16    	 4.47 s
epoch 1/1000 	 batch 25600/90000 	 loss 2.21 	 accuracy 0.19    	 5.11 s
epoch 1/1000 	 batch 28800/90000 	 loss 1.95 	 accuracy 0.28    	 5.80 s
epoch 1/1000 	 batch 32000/90000 	 loss 2.01 	 accuracy 0.31    	 6.43 s
epoch 1/1000 	 batch 35200/90000 	 loss 1.80 	 accuracy 0.34    	 6.99 s
epoch 1/1000 	 batch 38400/90000 	 loss 1.97 	 accuracy 0.25    	 7.53 s
epoch 1/1000 	 batch 41600/90000 	 loss 1.91 	 accuracy 0.19    	 8.14 s
epoch 1/1000 	 batch 44800/90000 	 loss 2.04 	 accurac

KeyboardInterrupt: 

In [ ]:
pv.utils.storage.save_parameters(model.state_dict(), "train19.model")

In [ ]:
m = Model()
m.load_state_dict(pv.utils.storage.load_parameters("train19.model"))

In [ ]:
m.eval()
X, Y = test_data.getdatas()
correct_tot = 0
for x, y in pv.data.data_generator(X, Y, batch_size=128, shuffle=False):
    y_pred = m(x)
    correct_tot += np.sum(y_pred.argmax(1, False).to_numpy() == y)
acc = correct_tot / Y.shape[0]
print(f"********** eval : accuracy {100*acc:.2f}% **********")

In [ ]:
epochs = range(1, len(acc_train) + 1)

plt.plot(epochs, acc_train, label="Train", marker=".")
plt.plot(epochs, acc_test, label="Test", marker=".")

plt.title("Accuracy over Epochs")
plt.xlabel("Epoch")
plt.ylabel("Accuracy")
plt.grid(True)
plt.legend()
# plt.ylim((0,1))
plt.tight_layout()

plt.savefig("train161.png")
plt.show()